In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Embedding, Dense, Flatten, Input, Concatenate, GlobalAveragePooling1D, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras import backend as K

# Load data
whole_data = pd.read_excel('data/whole_data_cleaned_batur.xlsx')
whole_data.drop(["website", "place_links", "description"], axis=1, inplace=True)

# Preprocess whole_data: Convert 'tags' from comma-separated strings to lists of integers
def safe_int_convert(tag_list):
    if isinstance(tag_list, list):
        return tag_list
    elif isinstance(tag_list, (int, float)):
        return [int(tag_list)]
    elif isinstance(tag_list, str):
        try:
            return list(map(int, tag_list.split(',')))
        except ValueError:
            return []
    return []

whole_data['tags'] = whole_data['tags'].apply(safe_int_convert)

# Convert rating to float and handle any missing data
whole_data['rating'] = whole_data['rating'].astype(float).fillna(whole_data['rating'].mean())

# Drop unnecessary columns
whole_data.drop('territory_id', axis=1, inplace=True)

# Pad the sequences for tags
tags_padded = pad_sequences(whole_data['tags'], padding='post')
whole_data['tags'] = list(tags_padded)

# Extract unique tags
tags_flat = [tag for sublist in whole_data['tags'].tolist() for tag in sublist]
unique_tags = np.unique(tags_flat)

# Define the model
def create_model(unique_tags, embedding_dim=64, dense_units=128):
    # Tag embedding
    tag_input = Input(shape=(None,), dtype=tf.int32, name='tags')
    tag_lookup = tf.keras.layers.IntegerLookup(vocabulary=unique_tags, mask_token=None)
    tag_embedding = Embedding(input_dim=len(unique_tags) + 1, output_dim=embedding_dim)
    tag_embeddings = tag_embedding(tag_lookup(tag_input))
    tag_embeddings = GlobalAveragePooling1D()(tag_embeddings)
    
    # Normalize rating
    rating_input = Input(shape=(1,), dtype=tf.float32, name='rating')
    rating_normalization = Normalization(axis=None)
    
    # Manually specify input shape
    rating_normalization.adapt(np.array([[whole_data['rating'].mean()]]))
    
    rating_normalized = rating_normalization(rating_input)
    
    # Concatenate all features
    combined_embeddings = Concatenate()([tag_embeddings, rating_normalized])
    
    dense = Dense(dense_units, activation='relu')(combined_embeddings)
    output = Dense(embedding_dim)(dense)
    
    model = Model(inputs=[tag_input, rating_input], outputs=output)
    return model

# Adjusted hyperparameters
model = create_model(unique_tags, embedding_dim=64, dense_units=128)

def triplet_loss(anchor, positive, negative, margin=1.0):
    positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(positive_distance - negative_distance + margin, 0.0)
    return tf.reduce_mean(loss)

# Create a dataset for triplet loss training
def generate_triplets(dataframe):
    anchor_features = {'tags': [], 'rating': []}
    positive_features = {'tags': [], 'rating': []}
    negative_features = {'tags': [], 'rating': []}
    
    for _, row in dataframe.iterrows():
        anchor_features['tags'].append(row['tags'])
        anchor_features['rating'].append(row['rating'])
        
        positive_idx = np.random.choice(dataframe.index)
        negative_idx = np.random.choice(dataframe.index)
        
        positive_row = dataframe.loc[positive_idx]
        negative_row = dataframe.loc[negative_idx]
        
        positive_features['tags'].append(positive_row['tags'])
        positive_features['rating'].append(positive_row['rating'])
        
        negative_features['tags'].append(negative_row['tags'])
        negative_features['rating'].append(negative_row['rating'])
    
    return (
        {k: pad_sequences(v, padding='post') if k == 'tags' else np.array(v) for k, v in anchor_features.items()},
        {k: pad_sequences(v, padding='post') if k == 'tags' else np.array(v) for k, v in positive_features.items()},
        {k: pad_sequences(v, padding='post') if k == 'tags' else np.array(v) for k, v in negative_features.items()}
    )

anchor_features, positive_features, negative_features = generate_triplets(whole_data)

def triplet_generator(anchor_features, positive_features, negative_features, batch_size=32):
    while True:
        indices = np.arange(len(anchor_features['tags']))
        np.random.shuffle(indices)
        
        for start in range(0, len(anchor_features['tags']), batch_size):
            end = start + batch_size
            batch_indices = indices[start:end]
            
            anchor_batch = {k: v[batch_indices] for k, v in anchor_features.items()}
            positive_batch = {k: v[batch_indices] for k, v in positive_features.items()}
            negative_batch = {k: v[batch_indices] for k, v in negative_features.items()}
            
            yield (anchor_batch, positive_batch, negative_batch)

batch_size = 32
triplet_gen = triplet_generator(anchor_features, positive_features, negative_features, batch_size=batch_size)

# Compile the model with custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(anchor_batch, positive_batch, negative_batch):
    with tf.GradientTape() as tape:
        anchor_embeddings = model(anchor_batch, training=True)
        positive_embeddings = model(positive_batch, training=True)
        negative_embeddings = model(negative_batch, training=True)
        
        loss = triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
epochs = 20
steps_per_epoch = len(anchor_features['tags']) // batch_size

for epoch in range(epochs):
    for step in range(steps_per_epoch):
        anchor_batch, positive_batch, negative_batch = next(triplet_gen)
        loss = train_step(anchor_batch, positive_batch, negative_batch)
        
        if step % 100 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.numpy()}")
            
# Save the trained model
model.save('triplet_model.keras')

model = tf.keras.models.load_model('triplet_model.keras', custom_objects={'triplet_loss': triplet_loss})

# Generate embeddings for all locations
features = {
    'tags': np.array(pad_sequences(whole_data['tags'], padding='post')),
    'rating': np.array(whole_data['rating'])
}
location_embeddings = model.predict(features)

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(location_embeddings)

# Function to generate recommendations for a specific place
def recommend(input_features, k=20):
    input_dict = {
        'tags': tf.convert_to_tensor([input_features['tags']], dtype=tf.int32),
        'rating': tf.convert_to_tensor([input_features['rating']], dtype=tf.float32)
    }
    
    # Generate the query embedding
    query_embedding = model.predict(input_dict)
    location_embeddings = model.predict(features)
    
    # Compute cosine similarity between the query embedding and all location embeddings
    similarities = cosine_similarity(query_embedding, location_embeddings)
    
    # Get the top-k most similar locations
    top_k_indices = similarities[0].argsort()[-k:][::-1]
    
    return top_k_indices

# Example usage
input_features = {
    'tags': [1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16],  # Example tags
    'rating': 4.5
}

recommendations = recommend(input_features, k=20)
print("Recommended items:", recommendations)

# Fetching the items from the dataset
recommended_items = whole_data.iloc[recommendations]
recommended_items


Epoch 1, Step 0, Loss: 3139170729984.0
Epoch 2, Step 0, Loss: 783682174976.0
Epoch 3, Step 0, Loss: 307921321984.0
Epoch 4, Step 0, Loss: 169230303232.0
Epoch 5, Step 0, Loss: 147417825280.0
Epoch 6, Step 0, Loss: 49037533184.0
Epoch 7, Step 0, Loss: 66230501376.0
Epoch 8, Step 0, Loss: 18739812352.0
Epoch 9, Step 0, Loss: 39471996928.0
Epoch 10, Step 0, Loss: 14305816576.0
Epoch 11, Step 0, Loss: 12691216384.0
Epoch 12, Step 0, Loss: 8109365760.0
Epoch 13, Step 0, Loss: 3333383936.0
Epoch 14, Step 0, Loss: 1268987648.0
Epoch 15, Step 0, Loss: 895018752.0
Epoch 16, Step 0, Loss: 384196608.0
Epoch 17, Step 0, Loss: 386181376.0
Epoch 18, Step 0, Loss: 90429800.0
Epoch 19, Step 0, Loss: 71172400.0
Epoch 20, Step 0, Loss: 56164032.0
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Recommended items: [ 54  67 138  58 120  66  68  72  74 141 186 187  44 136 134 133 131  97
 104 106]


,name,rating,tags,territory_id.1,locationYX
54,Hababam Class Museum,4.0,"[3, 4, 6, 0, 0, 0, 0]",1,"41.01375885653159, 29.04271916913588"
67,KidZania,4.3,"[18, 20, 0, 0, 0, 0, 0]",1,"41.000639922916086, 29.055213157621235"
138,Old Soup Kitchen Mosque,4.5,"[1, 2, 3, 5, 0, 0, 0]",3,"41.021785573263415, 28.954926399999994"
58,Historical Plane Tree of Çengelköy,4.3,"[1, 2, 3, 12, 13, 14, 15]",1,"41.05022883927636, 29.05278206286947"
120,Taksim Maksemi,4.4,"[1, 7, 0, 0, 0, 0, 0]",0,"41.03720659644188, 28.98437188534821"
66,Emaar Aquarium and Underwater Zoo,4.3,"[18, 20, 21, 0, 0, 0, 0]",1,"41.00262778665512, 29.072372921916998"
68,Istanbul Museum of Graphic Arts,4.4,"[2, 4, 6, 0, 0, 0, 0]",1,"41.00401771349877, 29.065356720589417"
72,Filizler Meatball Restaurant,4.0,"[13, 0, 0, 0, 0, 0, 0]",1,"41.02255261847614, 29.00690662144621"
74,Pasalimanı Restaurant,4.0,"[13, 0, 0, 0, 0, 0, 0]",1,"41.03370626211392, 29.024870301579618"
141,Sarachane Archaeology Park,4.3,"[2, 3, 6, 11, 0, 0, 0]",3,"41.02221916611512, 28.95463060863906"


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Embedding, Dense, Flatten, Input, Concatenate, GlobalAveragePooling1D, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras import backend as K

# Load data
whole_data = pd.read_excel('data/whole_data_cleaned.xlsx')
whole_data.drop(["website","place_links","description"],axis=1,inplace=True)

# Preprocess whole_data: Convert 'tags' from comma-separated strings to lists of integers
def safe_int_convert(tag_list):
    if isinstance(tag_list, list):
        return tag_list
    elif isinstance(tag_list, (int, float)):
        return [int(tag_list)]
    elif isinstance(tag_list, str):
        try:
            return list(map(int, tag_list.split(',')))
        except ValueError:
            return []
    return []

whole_data['tags'] = whole_data['tags'].apply(safe_int_convert)

# Convert rating to float and handle any missing data
whole_data['rating'] = whole_data['rating'].astype(float).fillna(whole_data['rating'].mean())

# Drop unnecessary columns
whole_data.drop('territory_id', axis=1, inplace=True)

# Pad the sequences for tags
tags_padded = pad_sequences(whole_data['tags'], padding='post')
whole_data['tags'] = list(tags_padded)
# Extract unique tags
tags_flat = [tag for sublist in whole_data['tags'].tolist() for tag in sublist]
unique_tags = np.unique(tags_flat)

# Define the model
def create_model(unique_tags, embedding_dim=64, dense_units=128):
    # Tag embedding
    tag_input = Input(shape=(None,), dtype=tf.int32, name='tags')
    tag_lookup = tf.keras.layers.IntegerLookup(vocabulary=unique_tags, mask_token=None)
    tag_embedding = Embedding(input_dim=len(unique_tags) + 1, output_dim=embedding_dim)
    tag_embeddings = tag_embedding(tag_lookup(tag_input))
    tag_embeddings = GlobalAveragePooling1D()(tag_embeddings)
    
    # Normalize rating
    rating_input = Input(shape=(1,), dtype=tf.float32, name='rating')
    rating_normalization = Normalization(axis=None)
    
    # Manually specify input shape
    rating_normalization.build((None, 1))
    
    rating_normalized = rating_normalization(rating_input)
    
    # Concatenate all features
    combined_embeddings = Concatenate()([
        tag_embeddings,
        rating_normalized
    ])
    
    dense = Dense(dense_units, activation='relu')(combined_embeddings)
    output = Dense(embedding_dim)(dense)
    
    model = Model(inputs=[tag_input, rating_input], outputs=output)
    return model

# Adjusted hyperparameters
model = create_model(unique_tags, embedding_dim=64, dense_units=128)

def triplet_loss(anchor, positive, negative, margin=1.0):
    # Compute the distance between the anchor and the positive
    positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    
    # Compute the distance between the anchor and the negative
    negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    
    # Compute the triplet loss
    loss = tf.maximum(positive_distance - negative_distance + margin, 0.0)
    
    return tf.reduce_mean(loss)
# Create a dataset for triplet loss training
def generate_triplets(dataframe):
    anchor_features = {
        'tags': [],
        'rating': []
    }
    positive_features = {
        'tags': [],
        'rating': []
    }
    negative_features = {
        'tags': [],
        'rating': []
    }
    
    for _, row in dataframe.iterrows():
        anchor_features['tags'].append(row['tags'])
        anchor_features['rating'].append(row['rating'])
        
        positive_idx = np.random.choice(dataframe.index)
        negative_idx = np.random.choice(dataframe.index)
        
        positive_row = dataframe.loc[positive_idx]
        negative_row = dataframe.loc[negative_idx]
        
        positive_features['tags'].append(positive_row['tags'])
        positive_features['rating'].append(positive_row['rating'])
        
        negative_features['tags'].append(negative_row['tags'])
        negative_features['rating'].append(negative_row['rating'])
    
    return (
        {k: pad_sequences(v, padding='post') if k == 'tags' else np.array(v) for k, v in anchor_features.items()},
        {k: pad_sequences(v, padding='post') if k == 'tags' else np.array(v) for k, v in positive_features.items()},
        {k: pad_sequences(v, padding='post') if k == 'tags' else np.array(v) for k, v in negative_features.items()}
    )

anchor_features, positive_features, negative_features = generate_triplets(whole_data)

def triplet_generator(anchor_features, positive_features, negative_features, batch_size=32):
    while True:
        indices = np.arange(len(anchor_features['tags']))
        np.random.shuffle(indices)
        
        for start in range(0, len(anchor_features['tags']), batch_size):
            end = start + batch_size
            batch_indices = indices[start:end]
            
            anchor_batch = {k: v[batch_indices] for k, v in anchor_features.items()}
            positive_batch = {k: v[batch_indices] for k, v in positive_features.items()}
            negative_batch = {k: v[batch_indices] for k, v in negative_features.items()}
            
            yield (anchor_batch, positive_batch, negative_batch)

batch_size = 32
triplet_gen = triplet_generator(anchor_features, positive_features, negative_features, batch_size=batch_size)

# Compile the model with custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(anchor_batch, positive_batch, negative_batch):
    with tf.GradientTape() as tape:
        anchor_embeddings = model(anchor_batch, training=True)
        positive_embeddings = model(positive_batch, training=True)
        negative_embeddings = model(negative_batch, training=True)
        
        loss = triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
epochs = 20
steps_per_epoch = len(anchor_features['tags']) // batch_size

for epoch in range(epochs):
    for step in range(steps_per_epoch):
        anchor_batch, positive_batch, negative_batch = next(triplet_gen)
        loss = train_step(anchor_batch, positive_batch, negative_batch)
        
        if step % 100 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.numpy()}")
            
# Save the trained model
model.save('triplet_model.keras')

Epoch 1, Step 0, Loss: 1.0081740617752075
Epoch 2, Step 0, Loss: 0.9986446499824524
Epoch 3, Step 0, Loss: 1.004631757736206
Epoch 4, Step 0, Loss: 1.0032434463500977
Epoch 5, Step 0, Loss: 0.980785071849823
Epoch 6, Step 0, Loss: 1.0011537075042725
Epoch 7, Step 0, Loss: 0.9224637746810913
Epoch 8, Step 0, Loss: 1.0596095323562622
Epoch 9, Step 0, Loss: 0.9324941635131836
Epoch 10, Step 0, Loss: 0.9985631108283997
Epoch 11, Step 0, Loss: 0.7317497134208679
Epoch 12, Step 0, Loss: 0.8806846141815186
Epoch 13, Step 0, Loss: 0.8810694217681885
Epoch 14, Step 0, Loss: 0.9123201370239258
Epoch 15, Step 0, Loss: 0.8516880869865417
Epoch 16, Step 0, Loss: 0.7951695919036865
Epoch 17, Step 0, Loss: 0.9112540483474731
Epoch 18, Step 0, Loss: 0.8952871561050415
Epoch 19, Step 0, Loss: 0.6609818935394287
Epoch 20, Step 0, Loss: 0.8942583799362183


In [7]:
from tensorflow.keras.models import load_model

model = load_model('triplet_model.keras', custom_objects={'triplet_loss': triplet_loss})

# Generate embeddings for all locations
features = {
    'tags': np.array(pad_sequences(whole_data['tags'], padding='post')),
    'rating': np.array(whole_data['rating'])
}
location_embeddings = model.predict(features)

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(location_embeddings)

# Function to generate recommendations for a specific place
def recommend(input_features, k=20):
    input_dict = {
        'tags': tf.convert_to_tensor([input_features['tags']], dtype=tf.int32),
        'rating': tf.convert_to_tensor([input_features['rating']], dtype=tf.float32)
    }
    
    # Generate the query embedding
    query_embedding = model.predict(input_dict)
    location_embeddings = model.predict(features)
    
    # Compute cosine similarity between the query embedding and all location embeddings
    similarities = cosine_similarity(query_embedding, location_embeddings)
    
    # Get the top-k most similar locations
    top_k_indices = similarities[0].argsort()[-k:][::-1]
    
    return top_k_indices

# Example usage
input_features = {
    'tags': [15],  # Example tags
    'rating': 4.8
}

recommendations = recommend(input_features, k=30)
print("Recommended items:", recommendations)

# Fetching the items from the dataset
recommended_items = whole_data.iloc[recommendations]
print(recommended_items)

#fetching the items from the dataset
whole_data.iloc[recommendations]

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Recommended items: [ 58 159 158  45  52 110  89  91 156 166 174 114 162 170  82 186 137  85
 125 182 142 146 123 127 129  61  26 180 144  83]
                                                  name  rating  \
58                  Historical Plane Tree of Çengelköy     4.3   
159                                 Sultanahmet Square     4.6   
158                                     Topkapı Palace     4.7   
45                                        Maiden Tower     4.7   
52                                   Beylerbeyi Palace     4.7   
110                                       Galata Tower     4.7   
89                                      Çırağan Palace     4.7   
91                                    Ihlamur Pavilion     4.6   
156                                   Basilica Cistern     4.9   
166                    Turkish and Islamic Arts Museum     4.6   
174  Istanbul Mus

,name,rating,tags,territory_id.1,locationYX
58,Historical Plane Tree of Çengelköy,4.3,"[1, 2, 3, 12, 13, 14, 15]",1,"41.05022883927636, 29.05278206286947"
159,Sultanahmet Square,4.6,"[1, 2, 3, 12, 13, 14, 15]",3,"41.00640399569341, 28.976145898680326"
158,Topkapı Palace,4.7,"[1, 2, 3, 4, 6, 7, 9]",3,"41.01718931703204, 28.984405485753218"
45,Maiden Tower,4.7,"[1, 2, 3, 4, 7, 0, 0]",1,"41.021167, 29.00425\t"
52,Beylerbeyi Palace,4.7,"[1, 2, 3, 4, 7, 0, 0]",1,"41.042612743417656, 29.040053076966498"
110,Galata Tower,4.7,"[1, 2, 3, 4, 7, 0, 0]",0,"41.02587501770054, 28.974178292324684"
89,Çırağan Palace,4.7,"[1, 2, 3, 4, 7, 0, 0]",0,"41.04387308576512, 29.015929923862068"
91,Ihlamur Pavilion,4.6,"[1, 2, 3, 4, 7, 0, 0]",0,"41.05195756244766, 29.001298861846962"
156,Basilica Cistern,4.9,"[1, 2, 3, 4, 7, 9, 0]",3,"41.0135359792602, 28.977187971940108"
166,Turkish and Islamic Arts Museum,4.6,"[2, 3, 4, 5, 6, 0, 0]",3,"41.01085761893604, 28.975403362743883"


# Collabrative Filtering

In [10]:
# Load user data
user_data = pd.read_excel('data/generated_user_data.xlsx')
user_data

,user_id,user_gender,place_id,continent,score,user_age
0,1,Male,563837,2,1,54
1,1,Male,776089,2,0,54
2,1,Male,207024,2,1,54
3,1,Male,711495,2,1,54
4,1,Male,463612,2,1,54
...,...,...,...,...,...,...
9995,1000,Male,871687,1,1,51
9996,1000,Male,950867,1,1,51
9997,1000,Male,761356,1,1,51
9998,1000,Male,723599,1,1,51


In [11]:
user_data[user_data['user_id'] == 1]

,user_id,user_gender,place_id,continent,score,user_age
0,1,Male,563837,2,1,54
1,1,Male,776089,2,0,54
2,1,Male,207024,2,1,54
3,1,Male,711495,2,1,54
4,1,Male,463612,2,1,54
5,1,Male,733989,2,1,54
6,1,Male,375467,2,1,54
7,1,Male,992964,2,1,54
8,1,Male,342528,2,1,54
9,1,Male,459627,2,1,54


In [12]:
# Preprocess user_data
user_data['user_id'] = user_data.index

# Extract unique user_ids and place_ids
unique_user_ids = user_data['user_id'].unique()
unique_place_ids = user_data['place_id'].unique()

# Create mappings for user_ids and place_ids
user_id_to_index = {user_id: index for index, user_id in enumerate(unique_user_ids)}
place_id_to_index = {place_id: index for index, place_id in enumerate(unique_place_ids)}

# Map user_ids and place_ids to indices
user_data['user_index'] = user_data['user_id'].map(user_id_to_index)
user_data['place_index'] = user_data['place_id'].map(place_id_to_index)

In [13]:
user_data

,user_id,user_gender,place_id,continent,score,user_age,user_index,place_index
0,0,Male,563837,2,1,54,0,0
1,1,Male,776089,2,0,54,1,1
2,2,Male,207024,2,1,54,2,2
3,3,Male,711495,2,1,54,3,3
4,4,Male,463612,2,1,54,4,4
...,...,...,...,...,...,...,...,...
9995,9995,Male,871687,1,1,51,9995,72
9996,9996,Male,950867,1,1,51,9996,85
9997,9997,Male,761356,1,1,51,9997,64
9998,9998,Male,723599,1,1,51,9998,101


In [14]:
from keras.src.layers import Multiply
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten, Dense
from tensorflow.keras.models import Model

def create_ncf_model(num_users, num_places, embedding_dim=50, hidden_layers=[64, 32, 16, 8]):
    # User input and embedding
    user_input = Input(shape=(1,), name='user_input')
    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)
    user_embedding = Flatten()(user_embedding)
    
    # Place input and embedding
    place_input = Input(shape=(1,), name='place_input')
    place_embedding = Embedding(input_dim=num_places, output_dim=embedding_dim, name='place_embedding')(place_input)
    place_embedding = Flatten()(place_embedding)
    
    # GMF part: element-wise product of user and place embeddings
    gmf_vector = Multiply()([user_embedding, place_embedding])
    
    # MLP part: concatenate user and place embeddings
    mlp_vector = Concatenate()([user_embedding, place_embedding])
    
    # Hidden layers for MLP
    for units in hidden_layers:
        mlp_vector = Dense(units, activation='relu')(mlp_vector)
        
    
    # Concatenate GMF and MLP parts
    combined_vector = Concatenate()([gmf_vector, mlp_vector])
    
    # Final prediction layer
    output = Dense(1, activation='sigmoid')(combined_vector)
    
    # Build and compile model
    model = Model(inputs=[user_input, place_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

num_users = len(unique_user_ids)
num_places = len(unique_place_ids)
embedding_dim = 50

ncf_model = create_ncf_model(num_users, num_places, embedding_dim)


In [28]:
# Prepare training data
user_indices = user_data['user_index'].values
place_indices = user_data['place_index'].values
interactions = user_data['score'].values  # Assuming binary interaction (0 or 1)

# Train the model
ncf_model.fit(
    x=[user_indices, place_indices],
    y=interactions,
    batch_size=32,
    epochs=30,
    validation_split=0.2
)

# Save the trained NCF model
ncf_model.save('ncf_model.h5')


Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 1.1991e-07 - val_accuracy: 0.5000 - val_loss: 2.1638
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 9.3820e-08 - val_accuracy: 0.4985 - val_loss: 2.1910
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 8.3268e-08 - val_accuracy: 0.4985 - val_loss: 2.2194
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 7.1132e-08 - val_accuracy: 0.4985 - val_loss: 2.2420
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 5.1832e-08 - val_accuracy: 0.4975 - val_loss: 2.2718
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 4.1073e-08 - val_accuracy: 0.4980 - val_loss: 2.2905
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 3.8213e-08 - val_accuracy: 0.4975 - val_loss: 2.3145
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - los

In [35]:
def ncf_recommend(user_id, model, place_data, user_id_to_index, k=10):
    user_index = user_id_to_index[user_id]
    user_vector = np.full((len(place_data),), user_index)
    place_indices = np.arange(len(place_data))
    
    # Predict scores for all places for the given user
    predictions = model.predict([user_vector, place_indices])
    top_k_indices = predictions.flatten().argsort()[-k:][::-1]
    
    # Get the recommended places
    recommended_places = place_data.iloc[top_k_indices]
    return recommended_places

# Example usage
user_id = 123 # Replace with the actual user_id
recommended_places = ncf_recommend(user_id, ncf_model, whole_data, user_id_to_index, k=10)
print("Recommended places for user:", user_id)
recommended_places


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Recommended places for user: 123


,name,rating,tags,territory_id.1,locationYX
187,Ruins of Philanthropos Church,4.5,"[1, 2, 3, 5, 0, 0, 0]",3,"41.02291936064259, 28.984498453867523"
58,Historical Plane Tree of Çengelköy,4.3,"[1, 2, 3, 12, 13, 14, 15]",1,"41.05022883927636, 29.05278206286947"
67,KidZania,4.3,"[18, 20, 0, 0, 0, 0, 0]",1,"41.000639922916086, 29.055213157621235"
66,Emaar Aquarium and Underwater Zoo,4.3,"[18, 20, 21, 0, 0, 0, 0]",1,"41.00262778665512, 29.072372921916998"
65,Small Camlica Grove,4.5,"[9, 10, 11, 13, 0, 0, 0]",1,"41.020454964073814, 29.064851103286273"
64,Nevmekan,4.3,"[13, 0, 0, 0, 0, 0, 0]",1,"41.02525737615866, 29.010374767967782"
63,Historical Kuzguncuk Houses,4.7,"[1, 2, 3, 7, 0, 0, 0]",1,"41.035635359473225, 29.029979104928746"
62,Mahpeyker Kösem Valide Sultan Mosque,4.6,"[1, 2, 3, 5, 0, 0, 0]",1,"41.01988706849905, 29.029304699004275"
61,Mihrimah Sultan Mosque,4.8,"[1, 2, 3, 5, 0, 0, 0]",1,"41.02687730617994, 29.016465244271846"
60,Mehmet Naci Akgöz Kite Museum,4.5,"[4, 6, 0, 0, 0, 0, 0]",1,"41.02392981011595, 29.013201953741532"


In [25]:
# Example usage
user_id = 10  # Replace with the actual user_id
recommended_places = ncf_recommend(user_id, ncf_model, whole_data, k=10)
print("Recommended places for user:", user_id)
print(recommended_places)

TypeError: ncf_recommend() missing 1 required positional argument: 'user_id_to_index'

In [18]:
user_data[user_data["user_id"] == 1231]


,user_id,user_gender,place_id,continent,score,user_age,user_index,place_index
1231,1231,Female,207141,4,1,53,1231,150


In [19]:
input_features

{'tags': [1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16], 'rating': 4.5}

In [20]:
# Example usage
input_features = {
    'tags': [1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16],  # Example tags
    'rating': 4.5
}
user_id = 0  # Replace with the actual user_id

recommended_items = weighted_hybrid_recommend(input_features, user_id, model, ncf_model, whole_data, user_id_to_index, cb_weight=0, cf_weight=0.9, k=10)

NameError: name 'weighted_hybrid_recommend' is not defined

In [21]:
print("Recommended items for user:", user_id)

Recommended items for user: 0


In [22]:
print(recommended_items)

                                                  name  rating  \
82                                   Dolmabahce Palace     4.7   
162                        Istanbul Archaeology Museum     4.6   
170                        Great Palace Mosaics Museum     4.4   
166                    Turkish and Islamic Arts Museum     4.6   
174  Istanbul Museum of the History of Science and ...     4.4   
186                                       Grand Bazaar     4.1   
156                                   Basilica Cistern     4.9   
45                                        Maiden Tower     4.7   
89                                      Çırağan Palace     4.7   
52                                   Beylerbeyi Palace     4.7   
110                                       Galata Tower     4.7   
159                                 Sultanahmet Square     4.6   
91                                    Ihlamur Pavilion     4.6   
160                                       Gulhane Park     4.7   
58        